In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

import pickle

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

from sklearn.base import clone

In [2]:
DATASET_PATH = "dataset/"

In [3]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [4]:
data_batch = unpickle(DATASET_PATH + 'data_batch_1')

X_train, y_train = data_batch[b'data'], np.asarray(data_batch[b'labels'])
print(X_train.shape, y_train.shape)
y_train[0]
#shuffle_index = np.random.permutation(9000)
#X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

(10000, 3072) (10000,)


6

In [5]:
# red_pixels = X_train[0,:1024]
# plt.imshow(red_pixels.reshape(32,32), cmap = plt.get_cmap('Reds'), interpolation="nearest")
# plt.axis("off")
# plt.show()

# green_pixels = X_train[0,1024:2048]
# plt.imshow(green_pixels.reshape(32,32), cmap = plt.get_cmap('Greens'), interpolation="nearest")
# plt.axis("off")
# plt.show()

# blue_pixels = X_train[0,2048:3072]
# plt.imshow(blue_pixels.reshape(32,32), cmap = plt.get_cmap('Blues'), interpolation="nearest")
# plt.axis("off")
# plt.show()

# print(y_train[0])

In [6]:
y_train_frog = (y_train==6)
print(y_train_frog.shape)

(10000,)


In [7]:
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_frog)

sgd_clf.predict([X_train[0]])

array([False], dtype=bool)

In [13]:
precision_list = list()
recall_list = list()
f1_list = list()

skfolds = StratifiedKFold(n_splits=3, random_state=42)

for train_index, test_index in skfolds.split(X_train, y_train_frog):
    print(train_index.shape)
    clone_clf = clone(sgd_clf)
    X_train_folds = X_train[train_index]
    y_train_folds = (y_train_frog[train_index])
    X_test_fold = X_train[test_index]
    y_test_fold = (y_train_frog[test_index])
    
    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print("Error: ", n_correct / len(y_pred)) 
    
#     print(confusion_matrix(y_test_fold, y_pred))

    precision_list.append(precision_score(y_test_fold, y_pred))
    recall_list.append(recall_score(y_test_fold, y_pred))
    f1_list.append(f1_score(y_test_fold, y_pred))

(6666,)
Error:  0.701859628074
(6667,)
Error:  0.894689468947
(6667,)
Error:  0.892589258926


In [15]:
def display_scores(measure, scores):
    print(measure, sum(scores)/float(len(scores)))
    
display_scores("precision", precision_list)
display_scores("recall", recall_list)
display_scores("f1", f1_list)

precision 0.369633890302
recall 0.341325400592
f1 0.245246355351


In [10]:
confusion_matrix(y_train_frog, y_train_pred)

print("Precision", precision_score(y_train_frog, y_train_pred))
print("Recall", recall_score(y_train_frog, y_train_pred))
print("f1", f1_score(y_train_frog, y_train_pred))

Precision 0.255628177197
Recall 0.341747572816
f1 0.292480265891
